## This is a supervised machine learning problem(classification as i have a target column and have to find yes or no)

In [ ]:
#  Import all the required libraries

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.imputer SimpleImputer #Replace with some other values(mean, median, mode)
from sklearn.preprocessing import StandardScalar,OneHotEncoder



df = pd.read_csv("dataset/Telco_Customer_Churn_lyst1769326950438.csv")



customerID          7043
gender              7043
SeniorCitizen       7043
Partner             7043
Dependents          7043
tenure              7043
PhoneService        7043
MultipleLines       7043
InternetService     7043
OnlineSecurity      7043
OnlineBackup        7043
DeviceProtection    7043
TechSupport         7043
StreamingTV         7043
StreamingMovies     7043
Contract            7043
PaperlessBilling    7043
PaymentMethod       7043
MonthlyCharges      7043
TotalCharges        7043
Churn               7043
dtype: int64